In [ ]:
from datetime import datetime, timezone
import os
import dotenv
import logging

# Setup logging
logging.basicConfig(level=logging.INFO)

# Load environment variables
dotenv.load_dotenv()

# Constants
MAX_RETRIES = int(os.environ.get("MAX_RETRIES", "3"))
RETRY_DELAY = int(os.environ.get("RETRY_DELAY", "5"))
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
AZURE_OPENAI_KEY = os.getenv("AZURE_OPENAI_KEY")
DEPLOYMENT_NAME = os.getenv("DEPLOYMENT_NAME")

# Model settings
TEMPERATURE = float(os.getenv("TEMPERATURE", 0.2))
TOP_P = float(os.getenv("TOP_P", 0.2))
MODEL_NAME = os.getenv("MODEL_NAME", "gpt-4o")


In [ ]:
import openai

def call_llm_service(prompt, response_format="text"):
    client = openai.AzureOpenAI(
        azure_endpoint=AZURE_OPENAI_ENDPOINT,
        api_key=AZURE_OPENAI_KEY,
        api_version=AZURE_OPENAI_API_VERSION,
    )

    messages = [{"role": "system", "content": prompt}]
    if response_format == "json":
        messages[0]["content"] += "\nPlease provide the response strictly in JSON format."

    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=messages,
        max_tokens=4096,
        temperature=TEMPERATURE,
        top_p=TOP_P
    )

    return response.choices[0].message.content.strip()


In [ ]:
import fitz  # PyMuPDF

def extract_pdf_file(file_path):
    content = ""
    try:
        with fitz.open(file_path) as pdf_doc:
            for page_num in range(len(pdf_doc)):
                page = pdf_doc.load_page(page_num)
                content += page.get_text()
        logging.info(f"Extracted PDF: {file_path}")
    except Exception as pdf_err:
        logging.error(f"Error extracting text from PDF: {pdf_err}")
    return content

def load_text_file(file_path):
    try:
        with open(file_path, "r", encoding="utf-8") as file:
            return file.read()
    except Exception as e:
        logging.error(f"Failed to read file {file_path}: {e}")
        return ""


In [ ]:
def generate_test_cases(context_prompt, user_story, reference_docs):
    combined_prompt = f"""
You are an expert QA engineer.

Context:
{context_prompt}

User Story:
{user_story}

Reference Documents:
{reference_docs}

Task:
Based on the above context, user story, and supporting documents, generate a comprehensive list of automated test cases.
Include edge cases, input validation, negative and positive paths.
Provide output in clear bullet points with expected results.
"""
    return call_llm_service(combined_prompt)


In [ ]:
# File paths - update as needed
context_path = ""
user_story_path = ""
reference_doc_path = ""

# Load content
context_prompt = load_text_file(context_path)
user_story = load_text_file(user_story_path)
reference_docs = extract_pdf_file(reference_doc_path)

# Generate test cases
test_case_output = generate_test_cases(context_prompt, user_story, reference_docs)

# Show result
print("Generated Test Cases:\n")
print(test_case_output)
